In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import pandas as pd
import numpy as np
from collections import defaultdict

os.environ['CUDA_VISIBLE_DEVICES']='4'
pd.set_option('display.max_colwidth', None)
sys.path.append('/workspace/computationally-undermining-arguments/scripts/')
sys.path.append('/workspace/computationally-undermining-arguments/src-py')
sys.path.append('/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/')

In [3]:
%autoreload
import interact
import utility
import json
from argparse import Namespace
from tabulate import tabulate

In [4]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [5]:
jo_df = pd.read_pickle("/workspace/ceph_data/argument-undermining/jo_data/gen_data.pickle")
test_df = jo_df[jo_df.split=='test']
valid_df  = jo_df[jo_df.split=='val']

In [7]:
testing_df = test_df.sample(1000)

In [8]:
testing_df['post'] = testing_df['post'].apply(lambda t: [x.strip().lower() for x in t])
testing_df['filtered_comment'] = testing_df.apply(lambda row: list(filter(lambda x: x.strip().lower() not in row['post'] or x.strip().lower() != row['title'].strip().lower(), row['comment_sents'])), axis=1)

In [9]:
#testing_df = pd.read_pickle('./testing_df.pkl')

In [10]:
args = Namespace(max_length=150, device='cuda:0', temperature=1.0, 
                 top_k=50, top_p=0.95, no_sample=False, min_length=100, premise_extra=False,
                 build_instance_version='v3'
)

# # title as context and counter is premise
# model1_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov18_15-44-59_gammaweb03_jo_title_premise/'
# # full post as context and counter is premise
# model2_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov18_22-09-41_gammaweb03_jo_full_post_premise/'
# # title as context and counter is full comment
# model3_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov18_15-28-49_gammaweb03_jo_title_full_counter/'
# # full post as context and counter is full comment
# model4_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov18_15-28-49_gammaweb03_jo_title_full_counter/'
# # title + full post as context and counter is premise
# model5_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov19_09-40-45_gammaweb03_jo_argument_premise/'
# # title + full post as context and counter is full comment
# model6_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov19_09-45-06_gammaweb03_jo_argument_full_counter/'
# # full post as context and counter is full comment

# model7_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec01_19-17-20_gammaweb03_jo_argument_full_counter_v2/'
# model8_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Nov30_17-15-04_gammaweb03_jo_argument_premise_v2/'
# baseline_model2_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec02_12-38-29_gammaweb03_jo_baseline_gpt/'

# model9_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec05_14-55-51_gammaweb03_jo_argument_full_counter_v3/'
# model10_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec05_15-14-30_gammaweb03_jo_argument_premise_v3/'

# model9_6epochs_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec08_09-51-21_gammaweb03_jo_argument_full_counter_v3_6epochs/'
# model10_6epochs_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec08_09-27-38_gammaweb03_jo_argument_premise_v3_6epochs/'

baseline_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-48-03_gammaweb03_jo_baseline_gpt/'
app_v2_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/'
app_v3_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-53-13_gammaweb03_my_approach_v3/'
app_v4_path = '/workspace/ceph_data/argument-undermining/models/gen_models/Dec21_17-10-34_gammaweb03_my_approach_v4/'

In [11]:
def expand_df(df, pred_clms, attacks_clm='premise_counter_premise_pairs', post_clm='post', comment_clm='comment_sents'):
    expanded_data = []
    for idx, row in df.iterrows():
        for counter_idx, counter in enumerate(row[attacks_clm]):
            data = [
                row['title'],
                row[post_clm],
                row[comment_clm],
                ' '.join(row['filtered_comment']),
                #row['target'][counter_idx],
                counter[0],
                counter[1]
            ]

            for clm in pred_clms:
                data.append(row[clm][counter_idx][1])
                
            expanded_data.append(data)

    expanded_df = pd.DataFrame(expanded_data, columns=['title', 'post', 'full_comment', 'filtered_comment', 'premise', 'gt_counter'] + baseline_preds_clms + pred_clms)
    
    return expanded_df

Predicting on baseline:

In [ ]:
testing_df = utility.pred_attacks(testing_df, baseline_path, args, 'baseline_final', context='title+full_post', baseline=True)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Some weights of the model checkpoint at /workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-48-03_gammaweb03_jo_baseline_gpt/ were not used when initializing OpenAIGPTLMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias', 'transformer.position_ids']
- This IS expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/interact.py:140: UserWarning: Warning: model generating

Predicting on random premises:

In [ ]:
#Choosing 3 random premises
testing_df['rand_premises'] = testing_df.apply(lambda row: list(np.random.choice(row['post'], 3, replace=True)), axis=1)

In [ ]:
args.build_instance_version = 'v2'
for i in range(0,3):
    testing_df = utility.pred_attacks(testing_df, app_v2_path, args, 'app_v2_rand_{}'.format(i), rand_premises_clm='rand_premises', rand_premise_idx=i, context='title+full_post', random=True)

args.build_instance_version = 'v3'
for i in range(0,3):
    testing_df = utility.pred_attacks(testing_df, app_v3_path, args, 'app_v3_rand_{}'.format(i), rand_premises_clm='rand_premises', rand_premise_idx=i, context='title+full_post', random=True)

args.build_instance_version = 'v4'
for i in range(0,3):
    testing_df = utility.pred_attacks(testing_df, app_v4_path, args, 'app_v4_rand_{}'.format(i), rand_premises_clm='rand_premises', rand_premise_idx=i, context='title+full_post', random=True)

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Some weights of the model checkpoint at /workspace/ceph_data/argument-undermining/models/gen_models/Dec21_17-10-34_gammaweb03_my_approach_v4/ were not used when initializing OpenAIGPTLMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias', 'transformer.position_ids']
- This IS expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/interact.py:115: UserWarning: Warning: Weak premise coul

Predicting on Title:

In [ ]:
args.build_instance_version = 'v2'
testing_df = utility.pred_attacks(testing_df, app_v2_path, args, 'app_v2_title', 
                                  rand_premises_clm='title', rand_premise_idx=None, context='title+full_post', random=True)

args.build_instance_version = 'v3'
testing_df = utility.pred_attacks(testing_df, app_v3_path, args, 'app_v3_title', 
                                  rand_premises_clm='title', rand_premise_idx=None, context='title+full_post', random=True)

In [ ]:
args.build_instance_version = 'v4'
testing_df = utility.pred_attacks(testing_df, app_v4_path, args, 'app_v4_title', 
                                  rand_premises_clm='title', rand_premise_idx=None, context='title+full_post', random=True)

Predicting on the weak premises:

In [ ]:
args.build_instance_version = 'v2'
testing_df = utility.pred_attacks(testing_df, app_v2_path, args, 'app_v2', context='title+full_post')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
Some weights of the model checkpoint at /workspace/ceph_data/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/ were not used when initializing OpenAIGPTLMHeadModel: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias', 'transformer.position_ids']
- This IS expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing OpenAIGPTLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/workspace/computationally-undermining-arguments/thirdparty/transfer-learning-conv-ai/interact.py:145: UserWarning: Warning: model generating 

In [ ]:
args.build_instance_version = 'v3'
testing_df = utility.pred_attacks(testing_df, app_v3_path, args, 'app_v3', context='title+full_post')

In [ ]:
args.build_instance_version = 'v4'
testing_df = utility.pred_attacks(testing_df, app_v4_path, args, 'app_v4', context='title+full_post')

In [ ]:
testing_df['weak_premises'] = testing_df.premise_counter_premise_pairs.apply(lambda x: len(x))
testing_df = testing_df[testing_df.weak_premises > 0]

In [37]:
expanded_df = expand_df(testing_df, ['app_v2', 'app_v3', 'app_v4', 'baseline_final']+['app_v3_rand_{}'.format(i) for i in range(3)]+['app_v2_rand_{}'.format(i) for i in range(3)]+['app_v4_rand_{}'.format(i) for i in range(3)]+['app_v2_title', 'app_v3_title', 'app_v4_title'])

In [38]:
post_counter_premises_df = expanded_df[['title', 'gt_counter']].groupby('title').agg({
    'gt_counter' : lambda gt_counter: list(gt_counter)
}).reset_index()
post_counter_premises_dict = pd.Series(post_counter_premises_df.gt_counter.values, index=post_counter_premises_df.title).to_dict()

post_counter_arguments_df = expanded_df[['title', 'filtered_comment']].groupby('title').agg({
    'filtered_comment' : lambda filtered_comment: list(filtered_comment)
}).reset_index()
post_counter_arguments_dict = pd.Series(post_counter_arguments_df.filtered_comment.values, index=post_counter_arguments_df.title).to_dict()

In [39]:
expanded_df['all_counter_premises'] = expanded_df.title.apply(lambda x: post_counter_premises_dict[x])
expanded_df['all_counter_arguments'] = expanded_df.title.apply(lambda x: post_counter_arguments_dict[x])

------

In [40]:
testing_df.columns

Index(['post_id', 'split', 'comment_id', 'title', 'post', 'comment',
       'n_sentences', 'direct_sents', 'all_4_sents', 'comment_sents',
       'premise_counter_premise_pairs', 'num_attacks', 'filtered_comment',
       'pred_attack8', 'baseline_attack_2', 'weak_premises', 'pred_attack7',
       'pred_attack9', 'pred_attack10', 'pred_attack10_1epoch',
       'pred_attack10_6epochs', 'pred_attack09_6epochs', 'baseline_final',
       'app_v2', 'app_v3', 'rand_premises', 'app_v2_rand_0', 'app_v2_rand_1',
       'app_v2_rand_2', 'app_v1_rand_0', 'app_v1_rand_1', 'app_v1_rand_2',
       'app_v3_rand_0', 'app_v3_rand_1', 'app_v3_rand_2', 'app_v2_title',
       'app_v3_title', 'app_v4_rand_0', 'app_v4_rand_1', 'app_v4_rand_2',
       'app_v4_title', 'app_v4'],
      dtype='object')

In [41]:
testing_df.to_pickle('./testing_df.pkl')

In [42]:
expanded_df.to_pickle('./expanded_df.pkl')

In [43]:
expanded_df.columns

Index(['title', 'post', 'full_comment', 'filtered_comment', 'premise',
       'gt_counter', 'app_v2', 'app_v3', 'app_v4', 'baseline_final',
       'app_v3_rand_0', 'app_v3_rand_1', 'app_v3_rand_2', 'app_v2_rand_0',
       'app_v2_rand_1', 'app_v2_rand_2', 'app_v4_rand_0', 'app_v4_rand_1',
       'app_v4_rand_2', 'app_v2_title', 'app_v3_title', 'app_v4_title',
       'all_counter_premises', 'all_counter_arguments'],
      dtype='object')

In [114]:
expanded_df.head()

,title,post,full_comment,filtered_comment,premise,gt_counter,app_v2,app_v3,baseline_final,app_v1_rand_0,app_v1_rand_1,app_v1_rand_2,app_v2_rand_0,app_v2_rand_1,app_v2_rand_2,all_counter_premises,all_counter_arguments
0,"""God is good."" doesn't mean or say anything. Also, that sentence, when taken to its logical conclusion in the context of the real world and the Bible, implies worshippers are hostages suffering Stockolm syndrome.","[obligatory 'this probably applies to other religions but i'm focusing on christianity because i know more about it. ', like many agnostic atheists, one of my qualms with many god claims is the problem of evil., i, personally, think it's an extremely good argument against a tri omni god., usually, i find the response is questioning my perception of 'good' and saying 'good' is whatever god says it is., if you agree with this, my point is, this does not connect at all to our perception of good., if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment ., i'll be more specific if you actually want to address this last point, i more just typed it to illustrate my perception that if god is good, and wanted people to use the human word 'good' or it's translations to describe him, he would have actually wanted to imply that it describes him., also if you do not think his sense of 'good' is the same as yours, there is no reason to accept him as the definition other than here comes the hostage comparison because he's more powerful and says he'll punish us i know not all denominations agree on hell, but if you do this point is for you if we don't., also, it does not make sense to say god would use the word 'good' to describe himself, including translated versions of the word using the other language's version of that word, but not use the human understanding of it without clarifying what he means.]","[if you agree with this, my point is, this does not connect at all to our perception of good., if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment ., one of the common answers to this is that god has a logically sufficient reason to allow these all these horrible things, but that we as mere lowly humans in our current state are just unable to comprehend., it will all become clear in the afterlife., in other words, there is no logical incompatibility between the existence of evil and the existence of god, depending on your theology., this is not my view btw., i'm an atheist as well.]","if you agree with this, my point is, this does not connect at all to our perception of good. if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment . one of the common answers to this is that god has a logically sufficient reason to allow these all these horrible things, but that we as mere lowly humans in our current state are just unable to comprehend. it will all become clear in the afterlife. in other words, there is no logical incompatibility between the existence of evil and the existence of god, depending on your theology. this is not my view btw. i'm an atheist as well.","[if you agree with this, my point is, this does not connect at all to our perception of good., if it did, god would not allow insert horrible event that happens daily in real life and encourage insert horrible bible verse about slavery women war punishment .]","one of the common answers to this is that god has a logically sufficient reason to allow these all these horrible things, but that we as mere lowly humans in our current state are just unable to comprehend. it will all become clear in the afterlife. in other words, there is no logical incompatibility between the existence of evil and the existence of god, depending on your theology. 